In [67]:
from numpy import *
import pandas as pd
from PIL import Image
from copy import *


In [68]:
pixels_dict = {}
readRGB = []
solvedCat = []
def readGraynne(image_name):
    img = Image.open(image_name, 'r').convert('I')
    data = asarray(img)
    return data


def diffRight(readRGB):
    diff = empty(int(20971520/2))
    for index in range(len(readRGB)-1):
        diff[index] = readRGB[index] -  readRGB[index+1]
    diff[int(20971520/2)-1] = 0
    return diff


def diffUp(readRGB):
    diff = empty(int(20971520/2))
    for index in range(1024):
        diff[index] = 0
    for index in range(1024,len(readRGB)):
        diff[index] = readRGB[index] -  readRGB[index-1024]
    return diff

def diffDown(readRGB):
    diff = empty(int(20971520/2))
    for index in range(len(readRGB)-1024):
        diff[index] = readRGB[index] -  readRGB[index+1024]
    for index in range(len(readRGB)-1024,len(readRGB)):
        diff[index] = 0
    return diff


def diffLeft(readRGB):
    diff = empty(int(20971520/2))
    diff[0] = 0
    for index in range(1,len(readRGB)):
        diff[index] = readRGB[index] -  readRGB[index-1]
    return diff

In [69]:
for i in range(10):
    readRGB.append(reshape(readGraynne("fake_microstructure/Target_1/image_" + str(i) + ".png"),1048576))

for i in range(10):
    solvedCat.append(reshape(readGraynne("fake_microstructure/Target_1/p2mask_np_" + str(i) + ".png"),1048576))
    
readRGB = reshape(readRGB, int(20971520/2))
solvedCat = reshape(solvedCat,int(20971520/2))
    
pixelsFrame = pd.DataFrame(pixels_dict)
print(size(readRGB))
print(readRGB)



10485760
[58313 58562 58923 ... 39330 37379 36131]


In [70]:
#print(len(solvedCat))
#print(size(solvedCat))
#print(solvedCat)

In [71]:
pixelsFrame['originalRGB'] = readRGB
pixelsFrame['solved Cat'] = solvedCat
pixelsFrame['difRight'] = diffRight(readRGB)
#pixelsFrame['difUpRight'] = diffRight(readRGB, solvedCat)

pixelsFrame['difUp'] = diffUp(readRGB)
#pixelsFrame['difLeftUp'] =
pixelsFrame['difLeft'] = diffLeft(readRGB)
#pixelsFrame['difLeftDown'] =
pixelsFrame['difDown'] = diffDown(readRGB)
#ixelsFrame['difRightDown']

In [72]:
#pixelsFrame['difUpRight'].head(5)

In [73]:
pixelsFrame.head(4000) 

,originalRGB,solved Cat,difRight,difUp,difLeft,difDown
0,58313,65535,-249.0,0.0,0.0,-96.0
1,58562,65535,-361.0,0.0,249.0,-25.0
2,58923,65535,-388.0,0.0,361.0,42.0
3,59311,65535,-330.0,0.0,388.0,91.0
4,59641,65535,-219.0,0.0,330.0,117.0
5,59860,65535,-96.0,0.0,219.0,124.0
6,59956,65535,-2.0,0.0,96.0,117.0
7,59958,65535,50.0,0.0,2.0,105.0
8,59908,65535,66.0,0.0,-50.0,93.0
9,59842,65535,64.0,0.0,-66.0,82.0
